# OpenNeuro Dataset Query by NIDM-Terms Example

In [198]:
import ipywidgets as widgets
import json
import glob, os
from os.path import join,basename
from IPython.display import display
from cognitiveatlas.api import get_concept, get_disorder
import requests


In [ ]:
# ask user for scicrunch API key
user_key=''
while user_key == '':
    user_key=input("Please enter your scicrunch.org API key (see scicrunch.org->My Account->API Keys): ")

In [157]:
# set up uber jsonld dictionary
data={}
# for all jsonld documents in this repo, load them into a graph
for dataset in glob.glob("../../terms/OpenNeuro_BIDS_terms/**"):
    # set top-level data key to datset number
    data[basename(dataset)] = {}
    # loop through all jsonld files and get isAbouts
    for jsonldfile in glob.glob(join(dataset,"*.jsonld")):
        #load jsonld file
        #print("Loading jsonld file: %s for datset: %s" %(basename(jsonldfile),basename(dataset)))
        with open(jsonldfile) as f:
            data[basename(dataset)] = json.load(f)

    

In [158]:
# find all isAbout concepts in data dictionary
isAbout_terms=[]
for key,val in data.items():
    isAbout_terms.append([subval for subkey,subval in data[key].items() if "isAbout" in subkey])

In [159]:
# isAbout concept URLs don't have any human-readable labels at this point so let's go and try to retrieve them
# first let's get labels for all the cognitive atlas isAbout concepts from their API
isAbout_term_labels={}
for terms in isAbout_terms:
    for url in terms:     
        # retrieve label from cog atlas
        if "cognitiveatlas" in url:
            # parse out id of term and get using cog atlas python tool...
            id = url.rsplit('/',1)[0].rsplit('/',1)[1]
            # don't know if this is a concept or disorder so we'll try both
            tmp = get_concept(id=id,silent=True)
            isAbout_term_labels[url] = tmp.json['name']
        elif "interlex" in url:
            # get label for interlex terms
            # to query interlex we need to use an api key
            
            #print("interlex url")
            

In [191]:

# temporary variables for query
global currentQueryTerm, queryTerms
currentQueryTerm =''
queryTerms = []
# handler for drop down
def dropdown_eventhandler(change):
    if change['name'] == 'value' and (change['new'] != change['old']):
        # set currentQueryTerm to selection
        #print("current query term changed %s" %change['new'])
        global currentQueryTerm
        currentQueryTerm = change['new']
            
# add handlers for simple GUI
def btn_addConceptToQuery(obj):
    global currentQueryTerm, queryTerms
    #add to query Terms
    print("Added \'%s\' to query..." %currentQueryTerm)
    queryTerms.append(currentQueryTerm)
# handlers for buttons
def btn_reset(obj):
    global queryTerms
    queryTerms = []
    print("query terms reset: %s" %queryTerms)
def btn_doQuery(obj):
    global queryTerms
    matching_datasets=[]
    print("running OR query on datasets for terms: %s" %queryTerms)
    for term in queryTerms:
        # run query by looking for url matching queryTerms in isAbout_terms_labels
        # which has mapping between isAbout URL and it's label
        for isabout_key, isabout_value in isAbout_term_labels.items():
            # check if isAbout_terms_labels value is the term we're looking for
            if isabout_value == term:
                # sometimes we have more than 1 isAbout URL so loop through them looking
                # for a match wtih our query term URL

                for dataset,dataset_annotations in data.items():
                    for key,value in dataset_annotations.items():
                        #print("looking for isAbout match (%s,%s)" %([isabout_key],value))
                        if (key=='isAbout')and (isabout_key in value):
                            matching_datasets.append(dataset)
                           
    print("Matching datasets: %s" %matching_datasets)
            
        
    
    

In [192]:
# bring up a new selector for adding a concept to the query
queryTermSelector = widgets.Dropdown(options=list(isAbout_term_labels.values()))
queryTermSelector.observe(dropdown_eventhandler)
# create a little user interface to query using isAbout concepts
addConceptToQuery = widgets.Button(description='Add Query Concept')
addConceptToQuery.on_click(btn_addConceptToQuery)
doQuery = widgets.Button(description='run query')
doQuery.on_click(btn_doQuery)
resetQuery = widgets.Button(description='reset query')
resetQuery.on_click(btn_reset)
display(queryTermSelector)
display(addConceptToQuery)
display(doQuery)
display(resetQuery)

Dropdown(options=('sleep', 'response inhibition', 'working memory', 'naming', 'anxiety'), value='sleep')

Button(description='Add Query Concept', style=ButtonStyle())

Button(description='run query', style=ButtonStyle())

Button(description='reset query', style=ButtonStyle())

query terms reset: []
Added 'naming' to query...
running OR query on datasets for terms: ['naming']
Matching datasets: ['001486', '001894']
